# The impact of the day zero drought on vegetation on the Western Cape

#### To run this analysis you need access to google earth engine. If you do not have access please sign up for a free account [here](https://signup.earthengine.google.com/])

## Setup libraries

In [ ]:
import wget
import sys
sys.path.append("src")
import uuid
import shutil
from shutil import unpack_archive
import ee
import folium
import geopandas as gpd
import rtree
import numpy as np
import pandas as pd
import math
import datetime
import subprocess, glob
from osgeo import gdal
import os
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt
import altair as alt
#helper functions for usinng gee
from gee_modis_clean import getQABits, updateMultipleMask
from gee_linear_model import addDependents, addHarmonics, constructBandNames, predict_coeffs, diff_predict
from gee_folium import add_ee_layer

In [8]:
#download the data
url = 'https://storage.googleapis.com/day_zero/eg.zip'
filename = wget.download(url)
unpack_archive(filename)
os.remove(filename) 

In [ ]:
#Authenticate Earth Engine 
#NB!!! right click and open this link in a new tab
ee.Authenticate()

In [ ]:
# Initialize the Earth Engine module.
ee.Initialize()

In [ ]:
# Test if earth engine is working by printing metadata for a DEM dataset.
print(ee.Image('USGS/SRTMGL1_003').getInfo())

## Read in raw data and clip to regions of interest
We select 3 biomes - renosterveld, fynbos, and forest- and 2 agricultural land uses - irrigated vineyards and rainfed grains.  
Biomes are masked with layers of naturla vegetation remnants and protected areas. 
We then extract these land uses in three regions

In [ ]:
#vegmap2018
#http://bgis.sanbi.org/SpatialDataset/Detail/1674
vegmap = gpd.read_file('data/input/vegmap2018/vegmap2018_wc.shp')

#wc protected areas
#http://bgis.sanbi.org/SpatialDataset/Detail/649
wcpa = gpd.read_file('data/input/WCBSP_PA_2017/BSP_PA_2017.shp')
wcpa= wcpa[['Name','geometry']]

#wc natural
#from CapeNature. email: Therese Forsyth <tforsyth@capenature.co.za>
remnants = gpd.read_file('data/input/remnants/natrem.shp')
remnants = remnants[['OBJECTID','Condition','geometry']]

#wc crop types
#from WC department of Agricultre. email: FC Basson <FCBasson@elsenburg.com>
crops = gpd.read_file('data/input/wc_crops_2013/Crop_Census_2013.shp')
crops = crops[['FIELD_ID','CROPS','CR_SUM','DRY_IRR','geometry']]

#regions
#manually created regions of interest
regions = gpd.read_file('data/input/regions/regions.shp')

In [ ]:
#filter BIOREGION_
bregions = ['Northwest Fynbos Bioregion','Southern Fynbos Bioregion',\
            'Southwest Fynbos Bioregion','West Coast Renosterveld Bioregion',\
            'East Coast Renosterveld Bioregion','Zonal & Intrazonal Forests',\
           'Azonal Forests']
vegmap = vegmap[vegmap['BIOREGION_'].isin(bregions)]

#recode as biomes
dict = {'Northwest Fynbos Bioregion' : 'Fy', 'Southern Fynbos Bioregion' : 'Fy', \
        'Southwest Fynbos Bioregion' : 'Fy', \
        'West Coast Renosterveld Bioregion' : 'R','East Coast Renosterveld Bioregion' : 'R',\
        'Zonal & Intrazonal Forests' : 'Fo', 'Azonal Forests' : 'Fo'} 
vegmap['biome']= vegmap['BIOREGION_'].map(dict) 

vegmap = vegmap[['OBJECTID','BIOREGION_','Name_18', 'biome','geometry']]

#filter remnants
remnants = remnants[remnants['Condition']=='Natural']

#grapes
grapes = crops[(crops['CR_SUM']=='Grapes') & (crops['DRY_IRR']=='Irrigated')] 
grapes['biome']="V"
#wheat
wheat = crops[(crops['CR_SUM']=='Grains') & (crops['DRY_IRR']=='Dry land')]
wheat['biome']="G"

In [ ]:
#reproject
vegmap = vegmap.to_crs({'init': 'epsg:4326'})
remnants = remnants.to_crs({'init': 'epsg:4326'})
regions = regions.to_crs({'init': 'epsg:4326'})

In [ ]:
veg_r = gpd.overlay(wcpa, regions, how='intersection')
veg_r = gpd.overlay(remnants,veg_r, how='intersection')
veg_r = gpd.overlay(vegmap,veg_r, how='intersection')
grape_r = gpd.overlay(grapes, regions, how='intersection')
wheat_r = gpd.overlay(wheat, regions, how='intersection')

In [ ]:
#veg_r.to_file('veg_r.shp')
#grape_r.to_file('gr.shp')
#wheat_r.to_file('wr.shp')

In [ ]:
veg_r = veg_r[['OBJECTID_1','Name_18','biome','rname','geometry']]
grape_r = grape_r[['FIELD_ID','CROPS','biome','rname','geometry']]
wheat_r = wheat_r[['FIELD_ID','CROPS','biome','rname','geometry']]

dict= {"FIELD_ID":"OBJECTID_1", "CROPS": "Name_18", "biome":"biome", "rname":"rname", "geometry":"geometry"}
grape_r = grape_r.rename(columns=dict)
wheat_r = wheat_r.rename(columns=dict)

In [ ]:
all_r = veg_r.append(grape_r).append(wheat_r)
all_r['code'] = all_r['rname'] + '__' + all_r['biome']

In [ ]:
#dissolve
all_r = all_r.dissolve(by='code', aggfunc='first')
all_r['code'] = all_r.index

In [ ]:
#convert to utm so unit is meters
all_r_utm =all_r.to_crs({'init': 'epsg:32734'})
#buffer by 125 meters
all_r_utm['geometry'] = all_r_utm.geometry.buffer(-125)
#polyarea
all_r_utm["area"] = all_r_utm['geometry'].area
#remove empty
all_r_utm = all_r_utm[all_r_utm["area"]>0]
#return to WGS
all_r =all_r_utm.to_crs({'init': 'epsg:4326'})

In [ ]:
codedic = {'cape_metropole__Fo': 1,\
'cape_metropole__Fy': 2,\
'cape_metropole__G': 3,\
'cape_metropole__R':4,\
'cape_metropole__V':5,\
'jonaskop__Fo':6,\
'jonaskop__Fy': 7,\
'jonaskop__G': 8,\
'jonaskop__R': 9,\
'jonaskop__V': 10,\
'west_coast__Fo': 11,\
'west_coast__Fy': 12,\
'west_coast__G': 13,\
'west_coast__R': 14,\
'west_coast__V': 15,\
'western_overberg__Fo': 16,\
'western_overberg__Fy': 17,\
'western_overberg__G': 18,\
'western_overberg__V': 19,\
'western_overberg__R': 20}


In [ ]:
codedic2 = {'cape_metropole__Forest': 1,\
'cape_metropole__Fynbos': 2,\
'cape_metropole__Grains': 3,\
'cape_metropole__Rensoterveld':4,\
'cape_metropole__V':5,\
'jonaskop__Forest':6,\
'jonaskop__Fynbos': 7,\
'jonaskop__Grains': 8,\
'jonaskop__Rensoterveld': 9,\
'jonaskop__Vineyards': 10,\
'west_coast__Forest': 11,\
'west_coast__Fynbos': 12,\
'west_coast__Grains': 13,\
'west_coast__Rensoterveld': 14,\
'west_coast__Vineyards': 15,\
'western_overberg__Forest': 16,\
'western_overberg__Fynbos': 17,\
'western_overberg__Grains': 18,\
'western_overberg__Vineyards': 19,\
'western_overberg__Rensoterveld': 20}

In [ ]:
all_r['codeint']= all_r['code'].map(codedic) 

## Visualize the extracted areas

In [ ]:
fig, ax = plt.subplots(1, 1)
all_r.plot(column='code',
            ax=ax)

In [ ]:
## Write results to file

In [ ]:
all_r.to_file('data/output/all_r.shp')

### Upload to GEE
sure we could do thisusing a bunch of command line tools and google cloud storage, but it is easier for you to ust download `all_r.shp` and upload it manually to GEE

### The Earth engine bit

In [ ]:
# read in polygons and rasterize
area = ee.FeatureCollection("users/glennwithtwons/all_r")

areaRas = area \
  .filter(ee.Filter.notNull(['codeint'])) \
  .reduceToImage(properties = ['codeint'],reducer = ee.Reducer.first()) \
  .rename(['codeint'])

#create mask
areaMask = area \
  .filter(ee.Filter.notNull(['codeint'])) \
  .map(lambda feature: feature.set('flag', ee.Number(1))) \
  .reduceToImage(['flag'],ee.Reducer.first()) \
  .rename(['flag'])

In [ ]:
# The number of cycles per year to model.
harmonics = 1
index= 'NDVI'

# Make a list of harmonic frequencies to model.
# These also serve as band name suffixes.
harmonicFrequencies = ee.List.sequence(1, harmonics)
harmonicFrequencies_st = list(range(1,harmonics+1))

# Construct lists of names for the harmonic terms.
cosNames = constructBandNames('cos_', harmonicFrequencies_st)
sinNames = constructBandNames('sin_', harmonicFrequencies_st)

# Independent variables = intercept, time, and kernel mean
independents = ee.List(['constant','t']) \
  .cat(cosNames).cat(sinNames)

In [ ]:
#get data
col_tr = ee.ImageCollection('MODIS/006/MYD13Q1')\
  .merge(ee.ImageCollection('MODIS/006/MOD13Q1'))\
  .filterBounds(area)\
  .filterDate('2000-01-01', '2014-12-31')\
  .map(updateMultipleMask(index,areaMask)) \
  .select(index)\
  .map(addDependents) \
  .map(addHarmonics(harmonicFrequencies,cosNames,sinNames))

col_val = ee.ImageCollection('MODIS/006/MYD13Q1')\
  .merge(ee.ImageCollection('MODIS/006/MOD13Q1'))\
  .filterBounds(area)\
  .filterDate('2001-01-01', '2019-12-31')\
  .map(updateMultipleMask(index,areaMask)) \
  .select(index)\
  .map(addDependents) \
  .map(addHarmonics(harmonicFrequencies,cosNames,sinNames))
   

In [ ]:
# The dependent variable we are modeling.
dependents = ee.List([index])

#fit the regression
# The output of the regression reduction is a 4x1 array image.
harmonicTrend_tr = col_tr.select(independents.cat(dependents)).reduce(ee.Reducer.robustLinearRegression(independents.length(), 1))
# Turn the array image into a multi-band image of coefficients.
hTC = harmonicTrend_tr.select('coefficients').arrayProject([0]).arrayFlatten([independents])
#RMSE
hResid = harmonicTrend_tr.select('residuals').arrayFlatten([dependents])

In [ ]:
#make predictions on data using fitted model
#then transform imgCol to multiband img
col_val_fit = col_val \
      .map(predict_coeffs(independents,hTC))\
      .map(diff_predict(index))\
      .sort('system:time_start', False) \
      .map(lambda image: image.select(['fitted'])) \
      .toBands() \
      .toFloat()

col_val_raw = col_val \
      .sort('system:time_start', False) \
      .map(lambda image: image.select([index])) \
      .toBands() \
      .toFloat()

col_exp = areaRas \
  .toFloat() \
  .addBands(col_val_fit)\
  .addBands(col_val_raw)\
  .addBands(ee.Image.pixelLonLat())\
  .toFloat() 

In [ ]:
#dates and names of bands
ndname = col_exp.getInfo()
ndvi_id = pd.DataFrame(ndname.get('bands'))['id']

In [ ]:
ndvi_id

In [ ]:
#export task
task = ee.batch.Export.image.toDrive(image=col_exp,
                                     region=area.geometry(),
                                     description='ndras_all',
                                     scale=250)
task.start()

In [ ]:
#check on task
task.status()

## Visualize EE output
This might take a while so beware

In [ ]:
im = col_val_fit.select(['1_2019_11_09_fitted'])

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 8000,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']}

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-33.98,18.39], zoom_start=10)

# Add the ndvi to the map object.
my_map.add_ee_layer(im, vis_params, 'map')
#my_map.add_ee_layer(area, vis_params, 'map')
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Read exported data from GDrive
Again I could link my gdrive to copy over the file, but that is a waste of time. It is quicker tpp manually download and upload it here to the `output` folder

## Read in ndvi data and convert to dataframe

In [ ]:
#first we need to merge files togehter as ee can output big rasters as multiple files
dirpath = r"data/output/ndras"
search_criteria = "ndras*.tif"
q = os.path.join(dirpath, search_criteria)
nd_files = glob.glob(q)
nd_files

In [ ]:
#function to convert raster to df
def r2df(filename):
    with rio.open(filename) as src:
    #read image
        image= src.read()
        # transform image
        bands,rows,cols = np.shape(image)
        image1 = image.reshape (bands,rows*cols)
        #export  to df
        nddf = pd.DataFrame(image1.transpose())
        nddf = nddf[pd.notnull(nddf[0])]
    return nddf

In [ ]:
#apply to all rasters
nddf = pd.concat([x for x in map(r2df, nd_files)], ignore_index=True)

In [ ]:
#rename column and subsample
nddf.columns = ndvi_id 
ndsamp = nddf.sample(frac=1).groupby('codeint').head(10)

#reshape df to long
ndsamp = pd.melt(ndsamp, id_vars=['longitude','latitude','codeint'],var_name = 'dt',value_name='vi')

#create new colun to id each row 
ndsamp['sat'] = ndsamp['dt'].str.slice(start=0,stop=1)
ndsamp['nd'] = ndsamp['dt'].str.slice(start=13)
#format dates
ndsamp['date'] = ndsamp['dt'].str.slice(start=2,stop=12)
ndsamp['date'] = pd.to_datetime(ndsamp['date'],format='%Y_%m_%d')
# rescale ndvi
ndsamp['vi'] = ndsamp['vi']/10000
#create test and val set
ndsamp['grp'] = ndsamp['date']>pd.to_datetime('2014-12-31')
#lat lng as string
ndsamp['px'] = ndsamp['longitude'].astype(str) +  ndsamp['latitude'].astype(str)

In [ ]:
#pivot and add a col for ndvi diff
ndsamp = pd.pivot_table(ndsamp,index=['longitude','latitude','codeint','date','grp','px'],columns='nd',values='vi')
ndsamp = ndsamp.reset_index()
ndsamp['residual'] = ndsamp['NDVI']-testn['fitted']
ndsamp = pd.melt(ndsamp, id_vars=['longitude','latitude','codeint','date','grp','px'],var_name = 'nd',value_name='vi')

In [ ]:
#calculate means
ndav = ndsamp.groupby(['codeint','nd','date','grp']).agg(vimean=('vi', 'mean')).reset_index()

In [ ]:
#to plot woth alt air, we need to combine to average data and the obs data
ndav['px'] = 'average'
ndav['vi'] = np.NaN

ndplot = ndsamp[ndsamp['nd']!='fitted']
ndplot['vimean'] = np.NaN
ndplot=ndplot[ndav.columns]
ndplot['fgp'] = "raw"
ndavplot = ndav
ndavplot['fgp'] = 'av'

ndall = ndplot.append(ndavplot)

In [ ]:
new_dict = {v: k for k, v in codedic2.items()}
ndall['codeint'] = ndall['codeint'].fillna(0).astype(int).map(new_dict)
#.map(codedic)

In [ ]:
#clean up labesl
ndall['nd'] = ndall['nd'].str.replace('NDVI', 'observed')
ndall['grp'] = np.where(ndall['grp'], 'prediction', 'calibration')

#create two dfs
#one for plotting obs and fitted
ndobs = ndall[ndall['nd']!='residual']

#onr for plotting anomalies
ndfit = ndall[ndall['nd']=='residual']

## Plotting!!

In [ ]:
alt.data_transformers.disable_max_rows()
#alt.renderers.enable('notebook')

In [ ]:
obs = alt.Chart(width=200,height=200).mark_line(
    opacity=0.3,
    size=0.2
).encode(
    x=alt.X('date',title='date'),
    y=alt.X('vi',title='NDVI'),
    color='grp',
    detail ='px'
)
group=alt.Chart(width=200,height=200).mark_line(
    size=2
).encode(
    x=alt.X('date',title='date'),
    y=alt.X('vimean',title='NDVI'),
    color='nd',
    detail ='px'
)

chart_ndvi =alt.layer(obs,group).facet("codeint",columns=3,data=ndobs)

In [ ]:
obs = alt.Chart(width=200,height=200).mark_line(
    opacity=0.3,
    size=0.2
).encode(
    x=alt.X('date',title='date'),
    y=alt.X('vi',title='residual'),
    color='grp',
    detail ='px'
)
group=alt.Chart(width=200,height=200).mark_line(
    size=2
).encode(
    x=alt.X('date',title='date'),
    y=alt.X('vimean',title='residual'),
    detail ='px'
)

#alt.layer(group,obs)
chart_fit =alt.layer(obs,group).facet("codeint",columns=3,data=ndfit)

In [ ]:
chart_ndvi

In [ ]:
chart_fit